In [1]:
import math

ngrid = 80
pi = 4.0*math.atan(1.0)
# All in Bohr units
k = 1.24
xmin, ymin, zmin =-7.0,-7.0,-7.0
xmax, ymax, zmax = 7.0, 7.0, 7.0
dq = (xmax-xmin)/float(ngrid-1)

In [2]:
import math
import numpy as np
import para

def evaluate_sab (Rnuc):    
    S = math.exp(-para.k*Rnuc)*(1.0 + para.k*Rnuc + (1.0/3.0)*(para.k*Rnuc)**2)
    return S

def evaluate_1s (r):
    orb_s = para.k**(3/2)*math.sqrt(para.pi)*math.exp(-para.k*r)
    return orb_s

def compute_orbital (Nxyz,natoms,ndim):
    sigma_1s = {}
    x = para.xmin
    for ic in range (1,para.ngrid+1):
        y = para.ymin
        for jc in range (1,para.ngrid+1):
            z = para.zmin
            for kc in range (1,para.ngrid+1):
                
                Rnuc = [Nxyz[0][0]-Nxyz[1][0],Nxyz[0][1]-Nxyz[1][1],Nxyz[0][2]-Nxyz[1][2]]
                Rnuc = math.sqrt(np.dot(Rnuc,Rnuc))

                S = evaluate_sab (Rnuc)
                S = 1.0/math.sqrt(2.0*(1.0+S))

                ra = [x-Nxyz[0][0],y-Nxyz[0][1],z-Nxyz[0][2]]
                ra = math.sqrt(np.dot(ra,ra))
                rb = [x-Nxyz[1][0],y-Nxyz[1][1],z-Nxyz[1][2]]
                rb = math.sqrt(np.dot(rb,rb))

                orb_sa = evaluate_1s (ra)
                orb_sb = evaluate_1s (rb)

                sigma_1s[ic,jc,kc] = S*(orb_sa + orb_sb)
                z += para.dq
            y += para.dq
        x += para.dq

    return sigma_1s    

In [4]:
import math
import numpy as np
import para
import orb

def print_orbital(N1,N2,N3,psi,xo,yo,zo,Nxyz):
    file1 = open ("H2+.cube","w")

    file1.write ("Cube file for H2+ ion\n")
    file1.write ("Bonding molecular orbital\n")
    file1.write ("%5d%16.6f%16.6f%16.6f\n"%(np.shape(Nxyz)[0],xo,yo,zo))
    file1.write ("%5d%16.6f%16.6f%16.6f\n"%(para.ngrid,para.dq,0.0,0.0))
    file1.write ("%5d%16.6f%16.6f%16.6f\n"%(para.ngrid,0.0,para.dq,0.0))
    file1.write ("%5d%16.6f%16.6f%16.6f\n"%(para.ngrid,0.0,0.0,para.dq))
    file1.write ("%5d%16.6f%16.6f%16.6f%16.6f\n"%(1,1.0,Nxyz[0][0],Nxyz[0][1],Nxyz[0][2]))
    file1.write ("%5d%16.6f%16.6f%16.6f%16.6f\n"%(1,1.0,Nxyz[1][0],Nxyz[1][1],Nxyz[1][2]))

#   NVal is taken as 1 by default              
    for ic in range (1,N1+1):
        for jc in range (1,N2+1):
            for kc in range (1,N3+1):
                if kc%6!=0 and kc!=N3:
                    file1.write ("%13.5e"%(psi[ic,jc,kc]))
                else:
                    file1.write ("%13.5e\n"%(psi[ic,jc,kc]))
    file1.close()

In [3]:
import math
import numpy as np
import para
import orb
import cube

def main():
    filename = "H2plus.xyz"
    Nxyz = []
    sigma_1s = {}
    xyz = open(filename)
    natoms = int(xyz.readline())
    title = xyz.readline()
    c = 0
    for line in xyz:
        x,y,z = line.split()        
        Nxyz.append([float(x), float(y), float(z)])
        c += 1
        if c == natoms:
            break
    xyz.close()
    ndim = np.shape(Nxyz)[1]

    sigma_1s = orb.compute_orbital (Nxyz,natoms,ndim)

    cube.print_orbital (para.ngrid,para.ngrid,para.ngrid,sigma_1s,para.xmin,para.ymin,para.zmin,Nxyz)

main()